In [1]:
import pandas as pd

In [2]:
prado = pd.read_csv('prado.csv')

# Process supports

Drop na

In [3]:
prado = prado.dropna(subset=['technical_sheet_soporte'])

Aggregate the paper, canvas and wood supported workarts

In [4]:
prado.loc[:,'technical_sheet_soporte'] = prado.loc[:,'technical_sheet_soporte'].apply(lambda x: 'Papel' if x.startswith('Papel') else x)
# prado.loc[:,'technical_sheet_soporte'] = prado.loc[:,'technical_sheet_soporte'].apply(lambda x: 'Lienzo' if x.startswith('Lienzo') else x)
# prado.loc[:,'technical_sheet_soporte'] = prado.loc[:,'technical_sheet_soporte'].apply(lambda x: 'Tabla' if x.startswith('Tabla') else x)
# Aggregate Canvas and Wood
canvas_wood = ['Lienzo', 'Tabla']
prado.loc[prado['technical_sheet_soporte'].apply(lambda x: any(x.startswith(prefix) for prefix in canvas_wood)), 'technical_sheet_soporte'] = 'Tabla-Lienzo'

In [15]:
prado.technical_sheet_soporte.value_counts().sort_values(ascending=False)[:10]

technical_sheet_soporte
Papel                                      6434
Tabla-Lienzo                               4088
Marfil                                      149
Placa de vidrio                             127
Lámina de cobre                              82
Revestimiento mural trasladado a lienzo      44
Tejido                                       16
Cartón                                       14
Terciopelo                                   12
Porcelana                                    11
Name: count, dtype: int64

Filter to keep the three most frequent supports and then undersample to balance the dataset

In [26]:
# Filter the dataframe to keep only the first three supports
filtered_prado = prado[prado['technical_sheet_soporte'].isin(['Papel', 'Tabla-Lienzo'])]

# Undersample each support to the minimum count
undersampled_prado = filtered_prado.groupby('technical_sheet_soporte').apply(lambda x: x.sample(1000)).reset_index(drop=True)

C:\Users\leoac\AppData\Local\Temp\ipykernel_4004\329508867.py:5: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  undersampled_prado = filtered_prado.groupby('technical_sheet_soporte').apply(lambda x: x.sample(1000)).reset_index(drop=True)


# Create the dataset

Create a column for matching the filenames

In [27]:
import os
undersampled_prado['image_file_name'] = undersampled_prado.work_image_url.apply(lambda x: os.path.basename(x))
undersampled_prado['image_file_name']

0       41cf02a4-02be-4d65-9b79-ff2c4e2eb92b.jpg
1       d4b75e70-c254-4549-bda1-6afcb9b6e6f3.jpg
2       adb4e295-c839-4ed2-b8ab-3578d075ef59.jpg
3       5e840b25-5bc5-4524-88ce-59054e1a921b.jpg
4       2a754424-e920-49b1-b63c-c47e2f1ef55e.jpg
                          ...                   
1995    9f979477-b7df-4420-b22c-c817ef8776e3.jpg
1996    ba96347d-7116-4922-959f-d7d223d44f35.jpg
1997    7360ca6a-cde1-42f3-8a74-14fc6e707a71.jpg
1998    fab49aa5-8706-4988-9a54-a077b9bd58a3.jpg
1999    6f2c5525-e0c7-4d09-ba07-168d4a55c04b.jpg
Name: image_file_name, Length: 2000, dtype: object

Rename the support column and create the ultimate df with only filename and support type

In [28]:
undersampled_prado.rename({'technical_sheet_soporte': 'support'}, axis=1, inplace=True)
processed_prado = undersampled_prado[['image_file_name', 'support']]

In [29]:
f'{len(processed_prado)} images in the processed dataset'

'2000 images in the processed dataset'

Save a .feather file

In [30]:
processed_prado.to_feather('processed_prado.feather')